In [ ]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import time
import random
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np 

encryption_table = {}
last_encryption_key = None
first_encryption_done = False

encryption_times = []
decryption_times = []
repeated_encryption_times = []


def textuni(text):
    return [ord(char) for char in text]

def calculate_value(unicode_val, key):
    if key < unicode_val:
        return key - unicode_val
    elif key > unicode_val:
        return unicode_val - key
    else:
        return -key

def twos_comp(num):
    if num < 0:
        num_bits = num.bit_length() + 1
        twos_complement = (1 << num_bits) + num
        return twos_complement
    return num

def collatz_transform(value, iterations):
    for _ in range(iterations):
        if value % 2 == 0:
            value //= 2
        else:
            value = 3 * value + 1
    return value

def encrypt_text(text, key, iterations=2):
    global first_encryption_done
    
    unicode_values = textuni(text)
    encrypted_values = []
    
    random_offset = 0 if not first_encryption_done else random.randint(1, 100)

    for unicode_val in unicode_values:
        value = calculate_value(unicode_val, key) + random_offset
        value = twos_comp(value)
        transformed_value = collatz_transform(value, iterations)
        
        encrypted_char = chr((transformed_value % 94) + 33)
        encrypted_values.append(encrypted_char)

    encrypted_text = ''.join(encrypted_values)
    
    encryption_table[encrypted_text] = (text, random_offset)
    
    first_encryption_done = True
    
    return encrypted_text

def decrypt_text(encrypted_text, key):
    if encrypted_text in encryption_table:
        original_text, random_offset = encryption_table[encrypted_text]

        if key == last_encryption_key:
            del encryption_table[encrypted_text]
            return original_text
        else:
            return "Invalid Key"
    else:
        return None

def count_characters(text):
    letters = sum(1 for char in text if char.isalpha())
    digits = sum(1 for char in text if char.isdigit())
    special = len(text) - letters - digits
    return letters, digits, special

def update_counts():
    # Update character count for original text
    original_text = text_entry.get("1.0", tk.END).strip()
    char_count_label.config(text=f"Character Count (Original): {len(original_text)}")

    # Update character count for encrypted text
    encrypted_text = encrypted_text_box.get("1.0", tk.END).strip()
    letters, digits, special = count_characters(encrypted_text)
    char_count_encrypt_label.config(text=f"Letters: {letters}, Digits: {digits}, Special: {special}")

    # Update character count for decrypted text
    decrypted_text = decrypted_text_box.get("1.0", tk.END).strip()
    letters, digits, special = count_characters(decrypted_text)
    char_count_decrypt_label.config(text=f"Letters: {letters}, Digits: {digits}, Special: {special}")

def handle_encrypt():
    text = text_entry.get("1.0", tk.END).strip()
    key = key_entry.get()
    iterations = iterations_entry.get()

    if not text or not key or not iterations:
        messagebox.showerror("Error", "Please fill in all fields.")
        return

    try:
        key = int(key)
        iterations = int(iterations)

        start_time = time.perf_counter()
        encrypted_text = encrypt_text(text, key, iterations)
        end_time = time.perf_counter()

        encryption_time = end_time - start_time
        encryption_times.append(encryption_time)  # Store encryption time

        # Store repeated encryption times for the same input
        repeated_encryption_times.append(encryption_time)

        encrypted_text_box.delete("1.0", tk.END)
        encrypted_text_box.insert("1.0", encrypted_text)

        global last_encryption_key
        last_encryption_key = key

        digit_count_label.config(text=f"Total Count (Encrypted): {len(encrypted_text)}")
        encryption_time_label.config(text=f"Encryption Time: {encryption_time:.6f} seconds")

        # Update counts after encryption
        update_counts()

    except ValueError:
        messagebox.showerror("Error", "Key and Iterations must be numbers.")


def handle_decrypt():
    encrypted_text = encrypted_text_box.get("1.0", tk.END).strip()
    
    if not encrypted_text:
        messagebox.showerror("Error", "Please provide encrypted text for decryption.")
        return

    key = simpledialog.askinteger("Decryption Key", "Enter the decryption key:")
    
    if key is None:
        messagebox.showerror("Error", "Please enter a valid key.")
        return

    start_time = time.perf_counter()
    decrypted_text = decrypt_text(encrypted_text, key)
    end_time = time.perf_counter()

    decryption_time = end_time - start_time
    decryption_times.append(decryption_time)  # Store decryption time

    if decrypted_text:
        decrypted_text_box.delete("1.0", tk.END)
        decrypted_text_box.insert("1.0", decrypted_text)

        decryption_time_label.config(text=f"Decryption Time: {decryption_time:.6f} seconds")

        # Update counts after decryption
        update_counts()
    else:
        messagebox.showerror("Error", "Invalid encrypted text or decryption key.")

def show_time_graph():
    if not encryption_times or not decryption_times:
        messagebox.showerror("Error", "No data available to plot. Perform encryption and decryption first.")
        return

    graph_window = tk.Toplevel()
    graph_window.title("Encryption and Decryption Time Graph")
    
    fig, ax = plt.subplots(figsize=(8, 4))

    # Plot encryption and decryption times as a bar graph
    processes = ['Encryption', 'Decryption']
    times = [encryption_times[-1], decryption_times[-1]]  # Use the most recent times

    ax.bar(processes, times, color=['blue', 'red'])
    ax.set_xlabel('Process')
    ax.set_ylabel('Time (seconds)')
    ax.set_title('Time taken for Encryption and Decryption')

    canvas = FigureCanvasTkAgg(fig, master=graph_window)
    canvas.draw()
    canvas.get_tk_widget().pack()
    
def show_repeated_encryption_graph():
    if not repeated_encryption_times:
        messagebox.showerror("Error", "No repeated encryption data available. Encrypt the same text multiple times.")
        return

    avg_time = np.mean(repeated_encryption_times)  # Calculate the average time

    graph_window = tk.Toplevel()
    graph_window.title("Repeated Encryption Time Graph")

    fig, ax = plt.subplots(figsize=(8, 4))
    
    attempts = range(1, len(repeated_encryption_times) + 1)
    ax.bar(attempts, repeated_encryption_times, color='purple', label="Encryption Time")

    # Plot the average as a horizontal line
    ax.axhline(avg_time, color='red', linestyle='dashed', linewidth=2, label=f'Avg Time: {avg_time:.6f}s')

    ax.set_xlabel('Encryption Attempt')
    ax.set_ylabel('Time (seconds)')
    ax.set_title('Encryption Time for Same Input, Key, and Iterations')
    ax.legend()  # Show legend

    canvas = FigureCanvasTkAgg(fig, master=graph_window)
    canvas.draw()
    canvas.get_tk_widget().pack()


def create_gui():
    global text_entry, key_entry, iterations_entry, encrypted_text_box, decrypted_text_box
    global char_count_label, digit_count_label, encryption_time_label, decryption_time_label, char_count_decrypt_label, char_count_encrypt_label

    root = tk.Tk()
    root.title("Encrypt/Decrypt Tool")

    tk.Label(root, text="Text:").grid(row=0, column=0, padx=10, pady=10, sticky="nw")
    text_entry = tk.Text(root, width=70, height=10)
    text_entry.grid(row=0, column=1, padx=10, pady=10)

    tk.Label(root, text="Key:").grid(row=1, column=0, padx=10, pady=10)
    key_entry = tk.Entry(root, width=30)
    key_entry.grid(row=1, column=1, padx=10, pady=10)

    tk.Label(root, text="Iterations:").grid(row=2, column=0, padx=10, pady=10)
    iterations_entry = tk.Entry(root, width=30)
    iterations_entry.grid(row=2, column=1, padx=10, pady=10)

    encrypt_button = tk.Button(root, text="Encrypt", command=handle_encrypt, width=15, bg="lightblue")
    encrypt_button.grid(row=3, column=0, padx=10, pady=10)

    decrypt_button = tk.Button(root, text="Decrypt", command=handle_decrypt, width=15, bg="lightgreen")
    decrypt_button.grid(row=3, column=1, padx=10, pady=10)

    tk.Label(root, text="Encrypted Text:").grid(row=4, column=0, padx=10, pady=10, sticky="nw")
    encrypted_text_box = tk.Text(root, width=70, height=10)
    encrypted_text_box.grid(row=4, column=1, padx=10, pady=10)

    tk.Label(root, text="Decrypted Text:").grid(row=5, column=0, padx=10, pady=10, sticky="nw")
    decrypted_text_box = tk.Text(root, width=70, height=10)
    decrypted_text_box.grid(row=5, column=1, padx=10, pady=10)
    
    graph_button = tk.Button(root, text="Show Graph", command=show_time_graph, bg="lightyellow")
    graph_button.grid(row=3, column=4, padx=50, pady=10)  # Moved right

    char_count_label = tk.Label(root, text="Character Count (Original): 0")
    char_count_label.grid(row=0, column=2, padx=10, pady=5, sticky="nw")

    digit_count_label = tk.Label(root, text="Total Count (Encrypted): 0")
    digit_count_label.grid(row=1, column=2, padx=10, pady=5, sticky="nw")

    encryption_time_label = tk.Label(root, text="Encryption Time: 0.000000 seconds")
    encryption_time_label.grid(row=2, column=2, padx=10, pady=5, sticky="nw")

    decryption_time_label = tk.Label(root, text="Decryption Time: 0.000000 seconds")
    decryption_time_label.grid(row=3, column=2, padx=10, pady=5, sticky="nw")

    char_count_decrypt_label = tk.Label(root, text="Letters: 0, Digits: 0, Special: 0")
    char_count_decrypt_label.grid(row=5, column=2, padx=10, pady=5, sticky="nw")

    # New label for encrypted text character count
    char_count_encrypt_label = tk.Label(root, text="Letters: 0, Digits: 0, Special: 0")
    char_count_encrypt_label.grid(row=4, column=2, padx=10, pady=5, sticky="nw")
    
    repeated_graph_button = tk.Button(root, text="Repeated Encryption Graph", command=show_repeated_encryption_graph, bg="lightpink")
    repeated_graph_button.grid(row=4, column=4, padx=10, pady=10)


    root.mainloop()

if __name__ == "__main__":
    create_gui()